# Example of generation 

In [1]:
import torch
from config import get_config
from model import CDGPT
from tokenizer import SentencePieceTokenizer


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
torch.set_grad_enabled(False)

In [3]:
tokenizer_path = "checkpoints/tokenizer.model"
cfg = get_config()
cfg.tokenizer.path = tokenizer_path
tokenizer = SentencePieceTokenizer(tokenizer_path)

In [4]:
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord

fasta_file = "example.fasta"
parser = SeqIO.parse(fasta_file, "fasta")
dna_record = next(parser)
protein_record = next(parser)
reverse_translate_record = next(parser)

## Translation generation

In [4]:
model_path = "checkpoints/CD-GPT-1b.pth"
state = torch.load(model_path, map_location="cpu")
model = CDGPT(cfg)
model.load_state_dict(state["model"], strict=True)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.half().to(device).eval()

number of parameters: 1059.38M


CDGPT(
  (transformer): ModuleDict(
    (wte): Embedding(64000, 2304)
    (h): ModuleList(
      (0-11): 12 x Block(
        (attn): CasualSelfAttention(
          (c_attn): Linear(in_features=2304, out_features=6912, bias=False)
          (c_proj): Linear(in_features=2304, out_features=2304, bias=False)
        )
        (mlp): SwiGLU(
          (c_fc1): Linear(in_features=2304, out_features=6144, bias=False)
          (c_fc2): Linear(in_features=2304, out_features=6144, bias=False)
          (c_proj): Linear(in_features=6144, out_features=2304, bias=False)
        )
        (rms_1): RMSNorm()
        (rms_2): RMSNorm()
      )
    )
    (ln_f): RMSNorm()
  )
  (lm_head): Linear(in_features=2304, out_features=64000, bias=False)
)

In [6]:
prompt = f"<mRNA>{str(dna_record.seq)}</mRNA><translate><:>"
x = tokenizer.encode(prompt, eos=False, device=device)
output = model.generate(x,
                        max_new_tokens=128,
                        temperature=0.8,
                        top_k=128,
                        top_p=0.0,
                        stop_ids=(tokenizer.bos, tokenizer.eos, tokenizer.pad)
                        )
output = tokenizer.decode(output.sequences)

In [7]:
output = output[len(prompt):]
translate_res = output.split("</protein>")[0]

In [65]:
translate_gt = str(dna_record.seq.translate())
print(f"GROUND TRUTH ABOVE, GENERATION BELOW, MISMATCHES IN \033[91mRED\033[0m")
print(translate_gt)
for i in range(len(translate_gt)):
    if translate_res[i] == translate_gt[i]:
        print(translate_res[i], end="")
    else:
        print(f"\033[91m{translate_res[i]}\033[0m", end="")


GROUND TRUTH ABOVE, GENERATION BELOW, MISMATCHES IN RED
MSGLHRSSSAPMKKGLPPKELLDDLCSRFVLNVPKEDLESFERILFLVENAHWFYEDNSVERDPSLKSLTLKEFTSLIFNSCDVLKPYVPHLDDIFKDFTSYKVRVPVTGAIIL
MSGLHRSSSAAPMKKGLPPKELLDDLCSRFVLNVPKEDLESFERILFLVENAHWFYEDNSVERDPSLKSLTLKEFTSLIFNSCVVLKPYVPHLDDIFKDFTSYKVRVPVTGAII

## Reverse translation generation

In [5]:
# you can download this model from Tencent Weiyun Disk
model_path = "checkpoints/CD-GPT-1b-reverse-translation.pth"
state = torch.load(model_path, map_location="cpu")
model = CDGPT(cfg)
model.load_state_dict(state["model"], strict=True)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.half().to(device).eval()

number of parameters: 1059.38M


CDGPT(
  (transformer): ModuleDict(
    (wte): Embedding(64000, 2304)
    (h): ModuleList(
      (0-11): 12 x Block(
        (attn): CasualSelfAttention(
          (c_attn): Linear(in_features=2304, out_features=6912, bias=False)
          (c_proj): Linear(in_features=2304, out_features=2304, bias=False)
        )
        (mlp): SwiGLU(
          (c_fc1): Linear(in_features=2304, out_features=6144, bias=False)
          (c_fc2): Linear(in_features=2304, out_features=6144, bias=False)
          (c_proj): Linear(in_features=6144, out_features=2304, bias=False)
        )
        (rms_1): RMSNorm()
        (rms_2): RMSNorm()
      )
    )
    (ln_f): RMSNorm()
  )
  (lm_head): Linear(in_features=2304, out_features=64000, bias=False)
)

In [6]:
prompt = f"<protein>{str(protein_record.seq)}</protein><reverse_translate><:>"
x = tokenizer.encode(prompt, eos=False, device=device)
output = model.generate(x,
                        max_new_tokens=1024,
                        temperature=0.8,
                        top_k=128,
                        top_p=0.0,
                        stop_ids=(tokenizer.bos, tokenizer.eos, tokenizer.pad)
                        )
output = tokenizer.decode(output.sequences)

In [7]:
output = output[len(prompt):]
reverse_translate_res = output.split("</mRNA>")[0].split("<mRNA>")[-1]

In [9]:
reverse_translate_gt = str(reverse_translate_record.seq)
print(f"GROUND TRUTH ABOVE, GENERATION BELOW, MISMATCHES IN \033[91mRED\033[0m")
print(reverse_translate_gt)
for i in range(len(reverse_translate_gt)):
    if i >= len(reverse_translate_res):
        break
    if reverse_translate_res[i] == reverse_translate_gt[i]:
        print(reverse_translate_res[i], end="")
    else:
        print(f"\033[91m{reverse_translate_res[i]}\033[0m", end="")

GROUND TRUTH ABOVE, GENERATION BELOW, MISMATCHES IN RED
ATGATGTGTGAAGTGATGCCCACGATTAACGAGGACACCCCAATGAGCCAAAGGGGGTCCCAAAGCAGTGGCTCGGACTCAGACTCCCATTTTGAGCAGCTGATGGTGAATATGCTAGATGAAAGGGACCGTCTTCTAGACACCCTTCGGGAGACCCAAGAAAGCCTCTCACTTGCCCAGCAAAGACTGCAGGATGTCATCTATGACCGAGATTCACTCCAGCGACAGCTCAATTCAGCCCTGCCACAGGATATCGAATCCCTAACAGGAGGGCTGACTGGTTCTAAGGGGGCTGATCCGCCGGAATTTGCTGCTCTGACAAAAGAATTAAATGCCTGTAGGGAGCAACTTCTAGAAAAGGAAGAAGAAATTTCTGAACTTAAAGCTGAAAGAAACAACACAAGACTGTTACTGGAGCACTTGGAATGCCTGGTGTCCCGACATGAAAGGTCACTAAGAATGACAGTGGTGAAACGGCAAGCCCAGTCCCCCTCAGGAGTCTCCAGTGAGGTTGAGGTTCTCAAGGCACTGAAATCTTTGTTTGAGCACCACAAGGCCTTGGATGAAAAGGTAAGGGAGCGACTGAGGGTTTCTTTAGAAAGAGTCTCTGCACTGGAAGAAGAACTAGCTGCTGCTAATCAGGAGATTGTTGCCTTGCGTGAACAAAATGTTCATATACAAAGAAAAATGGCATCAAGTGAGGGGTCCACAGAATCAGAACATCTTGAAGGGATGGAACCTGGTCAGAAAGTCCATGAAAAGCGTTTGTCCAATGGCTCTATAGACTCAACTGATGAAACTAGTCAAATAGTCGAACTACAAGAATTGCTTGAAAAGCAAAACTATGAAATGGCTCAAATGAAAGAACGTTTAGCAGCCCTTTCTTCCCGAGTGGGGGAAGTGGAACAGGAAGCAGAGACAGCAAGAAAGGATCTCATTAAAAC